In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
next_layer = "silver"

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
# Dynamic select query to read the data for specific proc_date
dynamic_query = f""" CREATE OR REPLACE TEMP VIEW temp_rop_winplus_order AS SELECT *
FROM udp_wcm_pro.udp_wcm_bronze_pro.rop_winplus_order
WHERE ProcDate like '{str(proc_date).replace('-', '')}%'
"""

# Execute the query
spark.sql(dynamic_query)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_realtime.winplus_order_bill_header
(
  hash_id BIGINT,
  receipt_no STRING,
  calday DATE,
  pos_no STRING,
  store_id STRING,
  store_name STRING,
  sale_type STRING,
  member_type STRING,
  customer_name STRING,
  total_amount DOUBLE,
  total_amount_cash DOUBLE,
  total_amount_other DOUBLE,
  customer_phone STRING,
  proc_date TIMESTAMP
) 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'true',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'true'
)
""")

In [0]:
spark.sql(
    f"""
CREATE OR REPLACE TEMP VIEW view_winplus_order_bill_header as
WITH dat AS (
    SELECT
         ROW_NUMBER() OVER (PARTITION BY main_tbl.BillCode, main_tbl.PosCode, main_tbl.StoreId ORDER BY ProcDate DESC) rn,
         main_tbl.BillCode receipt_no,
         CASE
            WHEN try_to_timestamp(main_tbl.BillDate, 'MM/dd/yyyy HH:mm:ss') IS NOT NULL
              THEN TO_DATE(try_to_timestamp(main_tbl.BillDate, 'MM/dd/yyyy HH:mm:ss'))
            WHEN try_to_timestamp(main_tbl.BillDate, 'dd/MM/yyyy HH:mm:ss') IS NOT NULL
              THEN TO_DATE(try_to_timestamp(main_tbl.BillDate, 'dd/MM/yyyy HH:mm:ss'))
            WHEN try_to_timestamp(main_tbl.BillDate, 'dd/MM/yyyy') IS NOT NULL
              THEN TO_DATE(try_to_timestamp(main_tbl.BillDate, 'dd/MM/yyyy'))
            WHEN try_to_timestamp(main_tbl.BillDate, 'MM/dd/yyyy') IS NOT NULL
              THEN TO_DATE(try_to_timestamp(main_tbl.BillDate, 'MM/dd/yyyy'))
            ELSE NULL
         END AS calday,
         main_tbl.PosCode pos_no,
         main_tbl.StoreId store_id,
         main_tbl.StoreName store_name,
         main_tbl.SaleType sale_type,
         main_tbl.MemberType member_type,
         main_tbl.CustomerName customer_name,
         main_tbl.TotalAmountItem total_amount,
         main_tbl.TotalCashAmount total_amount_cash,
         main_tbl.TotalOtherAmount total_amount_other,
         main_tbl.CustomerPhone customer_phone,
         TO_TIMESTAMP(main_tbl.ProcDate, 'yyyyMMddHHmmss') AS proc_date
     FROM temp_rop_winplus_order main_tbl
 )
 SELECT
     farm_fingerprint(CONCAT( 
       IFNULL(CAST(receipt_no AS STRING), ""),
       IFNULL(CAST(pos_no AS STRING), ""),
       IFNULL(CAST(store_id AS STRING), "")
     )) HASH_ID,
     * EXCEPT (rn)
 FROM dat
 WHERE rn = 1   
"""
)

In [0]:
spark.sql(f"DELETE FROM {catalog_name}.udp_wcm_silver_realtime.winplus_order_bill_header WHERE RECEIPT_NO IN (SELECT RECEIPT_NO FROM view_winplus_order_bill_header)")

In [0]:
spark.sql(f"INSERT INTO {catalog_name}.udp_wcm_silver_realtime.winplus_order_bill_header SELECT * FROM view_winplus_order_bill_header")